In [1]:
# Install Kaggle library
!pip install -q kaggle

# Make a directory for Kaggle and move the kaggle.json file there
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Change the permissions of the file
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset from Kaggle
!kaggle datasets download -d mdkhurshidjahan01/figshare-brain-tumor-dataset

# Unzip the downloaded dataset
!unzip figshare-brain-tumor-dataset.zip -d brain_tumor_dataset

Dataset URL: https://www.kaggle.com/datasets/mdkhurshidjahan01/figshare-brain-tumor-dataset
License(s): Apache 2.0
 99% 433M/436M [00:22<00:00, 22.5MB/s]
100% 436M/436M [00:22<00:00, 20.3MB/s]
Archive:  figshare-brain-tumor-dataset.zip
  inflating: brain_tumor_dataset/Fig Share/Testing/glioma/image_103192.png  
  inflating: brain_tumor_dataset/Fig Share/Testing/glioma/image_104263.png  
  inflating: brain_tumor_dataset/Fig Share/Testing/glioma/image_105758.png  
  inflating: brain_tumor_dataset/Fig Share/Testing/glioma/image_106611.png  
  inflating: brain_tumor_dataset/Fig Share/Testing/glioma/image_107584.png  
  inflating: brain_tumor_dataset/Fig Share/Testing/glioma/image_111735.png  
  inflating: brain_tumor_dataset/Fig Share/Testing/glioma/image_114652.png  
  inflating: brain_tumor_dataset/Fig Share/Testing/glioma/image_115499.png  
  inflating: brain_tumor_dataset/Fig Share/Testing/glioma/image_121515.png  
  inflating: brain_tumor_dataset/Fig Share/Testing/glioma/image_135912.

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import regularizers

sns.set_style('whitegrid')
import warnings
warnings.filterwarnings('ignore')

# Paths to your training and testing data
train_data_path = '/content/brain_tumor_dataset/Fig Share/Training'
test_data_path = '/content/brain_tumor_dataset/Fig Share/Testing'

# Create dataframes for training and testing data
def create_dataframe(data_path):
    filepaths = []
    labels = []
    folds = os.listdir(data_path)
    for fold in folds:
        f_path = os.path.join(data_path, fold)
        filelists = os.listdir(f_path)
        for file in filelists:
            filepaths.append(os.path.join(f_path, file))
            labels.append(fold)
    Fseries = pd.Series(filepaths, name='filepaths')
    Lseries = pd.Series(labels, name='label')
    return pd.concat([Fseries, Lseries], axis=1)

train_df = create_dataframe(train_data_path)
test_df = create_dataframe(test_data_path)

# Split test data into validation and test sets
valid, test = train_test_split(test_df, train_size=0.5, shuffle=True, random_state=42)

img_size = (150, 150)
batch_size = 32

# ImageDataGenerators
tr_gen = ImageDataGenerator()
ts_gen = ImageDataGenerator()

train_gen = tr_gen.flow_from_dataframe(train_df, x_col='filepaths', y_col='label', target_size=img_size,
                                       class_mode='categorical', color_mode='rgb', shuffle=True, batch_size=batch_size)
valid_gen = ts_gen.flow_from_dataframe(valid, x_col='filepaths', y_col='label', target_size=img_size,
                                       class_mode='categorical', color_mode='rgb', shuffle=True, batch_size=batch_size)
test_gen = ts_gen.flow_from_dataframe(test, x_col='filepaths', y_col='label', target_size=img_size,
                                      class_mode='categorical', color_mode='rgb', shuffle=False, batch_size=batch_size)

img_shape = (img_size[0], img_size[1], 3)
classes = train_gen.class_indices
num_class = len(classes)

# Use EfficientNetB0 as the base model (Teacher Model)
teacher_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=img_shape, pooling='max')

teacher_model = Sequential([
    teacher_model,
    BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001),
    Dense(256, kernel_regularizer=regularizers.l2(0.016), activity_regularizer=regularizers.l1(0.006),
          bias_regularizer=regularizers.l1(0.006), activation='relu'),
    Dropout(rate=0.4, seed=75),
    Dense(num_class, activation='softmax')
])

teacher_model.compile(Adamax(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
teacher_model.summary()

class TestAccuracyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        test_loss, test_acc = teacher_model.evaluate(test_gen, verbose=0)
        print(f'\nTesting loss: {test_loss}, Testing accuracy: {test_acc}\n')

# Callback to save the best model
checkpoint = tf.keras.callbacks.ModelCheckpoint('best_teacher_model.h5', monitor='val_accuracy', save_best_only=True, mode='max')

Epochs = 30
history = teacher_model.fit(x=train_gen, epochs=Epochs, verbose=1, validation_data=valid_gen,
                            validation_steps=None, shuffle=False, callbacks=[TestAccuracyCallback(), checkpoint])

# Load the best model
best_teacher_model = tf.keras.models.load_model('best_teacher_model.h5')

# Evaluate the best model on training, validation, and test sets
train_score = best_teacher_model.evaluate(train_gen, steps=16, verbose=1)
valid_score = best_teacher_model.evaluate(valid_gen, steps=16, verbose=1)
test_score = best_teacher_model.evaluate(test_gen, steps=16, verbose=1)

print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('-' * 20)
print("Validation Loss: ", valid_score[0])
print("Validation Accuracy: ", valid_score[1])
print('-' * 20)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

Found 2449 validated image filenames belonging to 3 classes.
Found 307 validated image filenames belonging to 3 classes.
Found 307 validated image filenames belonging to 3 classes.
16705208/16705208 [==============================] - 2s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional  (None, 1280)              4049571   
 )                                                               
                                                                 
 batch_normalization (Batch  (None, 1280)              5120      
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 256)               327936    
                                                                 
 dropout (Dropout)           (None, 256)               0         
        

10/16 [=================>............] - ETA: 1s - loss: 0.2059 - accuracy: 0.9837

16/16 [==============================] - 3s 182ms/step - loss: 0.2059 - accuracy: 0.9837
Train Loss:  0.17308203876018524
Train Accuracy:  1.0
--------------------
Validation Loss:  0.19078335165977478
Validation Accuracy:  0.9934853315353394
--------------------
Test Loss:  0.20588593184947968
Test Accuracy:  0.9837133288383484


In [3]:
# Define custom KD loss
def knowledge_distillation_loss(y_true, y_pred, teacher_logits, temperature=3, alpha=0.1):
    y_true = tf.argmax(y_true, axis=1)
    teacher_probs = tf.nn.softmax(teacher_logits / temperature)
    student_probs = tf.nn.softmax(y_pred / temperature)
    kd_loss = tf.keras.losses.categorical_crossentropy(teacher_probs, student_probs)
    ce_loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    return alpha * kd_loss + (1 - alpha) * ce_loss

# Prepare teacher logits for KD
teacher_logits = best_teacher_model.predict(train_gen, verbose=1)

# Define the Student Model
student_model = Sequential([
    Conv2D(filters=16, kernel_size=(3,3), padding="same", activation=LeakyReLU(alpha=0.1), input_shape=img_shape),
    BatchNormalization(),
    Conv2D(filters=16, kernel_size=(3,3), padding="same", activation=LeakyReLU(alpha=0.1)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(filters=32, kernel_size=(3,3), padding="same", activation=LeakyReLU(alpha=0.1)),
    BatchNormalization(),
    Conv2D(filters=32, kernel_size=(3,3), padding="same", activation=LeakyReLU(alpha=0.1)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(filters=64, kernel_size=(3,3), padding="same", activation=LeakyReLU(alpha=0.1)),
    BatchNormalization(),
    Conv2D(filters=64, kernel_size=(3,3), padding="same", activation=LeakyReLU(alpha=0.1)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(filters=128, kernel_size=(3,3), padding="same", activation=LeakyReLU(alpha=0.1)),
    BatchNormalization(),
    Conv2D(filters=128, kernel_size=(3,3), padding="same", activation=LeakyReLU(alpha=0.1)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Flatten(),

    Dense(128, activation=LeakyReLU(alpha=0.1), kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(64, activation=LeakyReLU(alpha=0.1), kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(num_class, activation='softmax')
])

student_model.compile(Adamax(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Define a custom training loop
def train_student_model(student_model, teacher_logits, train_gen, valid_gen, epochs, temperature=3, alpha=0.1):
    teacher_logits = np.array(teacher_logits)
    batch_size = train_gen.batch_size

    # Ensure the same length for teacher_logits and train_gen samples
    num_batches = len(train_gen)
    teacher_logits = teacher_logits[:num_batches * batch_size]

    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        batch_count = 0
        for x_batch, y_batch in train_gen:
            with tf.GradientTape() as tape:
                student_logits = student_model(x_batch, training=True)
                # Get corresponding teacher logits for the current batch
                start_idx = batch_count * batch_size
                end_idx = start_idx + len(x_batch)
                batch_teacher_logits = teacher_logits[start_idx:end_idx]
                if batch_teacher_logits.shape[0] != student_logits.shape[0]:
                    continue  # Skip this batch if sizes do not match
                loss = knowledge_distillation_loss(y_batch, student_logits, batch_teacher_logits, temperature, alpha)
            grads = tape.gradient(loss, student_model.trainable_variables)
            student_model.optimizer.apply_gradients(zip(grads, student_model.trainable_variables))
            batch_count += 1

            if batch_count >= num_batches:
                break

        val_loss, val_acc = student_model.evaluate(valid_gen, verbose=0)
        print(f'Validation loss: {val_loss}, Validation accuracy: {val_acc}')

# Train the student model
train_student_model(student_model, teacher_logits, train_gen, valid_gen, epochs=30)

# Save the student model
student_model.save('student_model.h5')

# Evaluate the student model on the test set
test_score = student_model.evaluate(test_gen, steps=16, verbose=1)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

77/77 [==============================] - 25s 308ms/step
Epoch 1/30


Validation loss: 1.183118224143982, Validation accuracy: 0.5863192081451416
Epoch 2/30
Validation loss: 1.0929518938064575, Validation accuracy: 0.638436496257782
Epoch 3/30
Validation loss: 0.8782894611358643, Validation accuracy: 0.7850162982940674
Epoch 4/30
Validation loss: 0.8964662551879883, Validation accuracy: 0.7654722929000854
Epoch 5/30
Validation loss: 0.8899588584899902, Validation accuracy: 0.7719869613647461
Epoch 6/30
Validation loss: 0.715828537940979, Validation accuracy: 0.8664495348930359
Epoch 7/30
Validation loss: 1.6953420639038086, Validation accuracy: 0.7524430155754089
Epoch 8/30
Validation loss: 0.7872994542121887, Validation accuracy: 0.8566775321960449
Epoch 9/30
Validation loss: 0.7453528046607971, Validation accuracy: 0.8664495348930359
Epoch 10/30
Validation loss: 1.9524953365325928, Validation accuracy: 0.5309446454048157
Epoch 11/30
Validation loss: 0.908949077129364, Validation accuracy: 0.8045602440834045
Epoch 12/30
Validation loss: 0.63930362462997

16/16 [==============================] - 3s 153ms/step - loss: 0.6731 - accuracy: 0.9186
Test Loss:  0.6730861663818359
Test Accuracy:  0.9185667634010315
